In [1]:
# use machine learning to turn natural language into structured data using spaCy, scikit-learn, and rasa NLU.
# Understanding intents and entities


In [2]:
# Intent classification with regex I, looking for keywords
import re

keywords = {'greet': ['hello', 'hi', 'hey'], 
            'goodbye': ['bye', 'farewell'], 
            'thankyou': ['thank', 'tnx']}

responses = {'default': 'default message',
            'goodbye': 'goodbye for now',
            'greet': 'Hello you! :)',
            'thankyou': 'you are very welcome'}

# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, keys in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile('|'.join(keys))
    
# Print the patterns
print(patterns)


{'greet': re.compile('hello|hi|hey'), 'goodbye': re.compile('bye|farewell'), 'thankyou': re.compile('thank|tnx')}


In [3]:
# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]


In [4]:
# Entity extraction with regex

bot_template = "BOT : {0}"
user_template = "USER : {0}"

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile('name|call(ed)')
    # Create a pattern for finding capitalized words
    name_pattern = re.compile('[A-Z]{1}[a-z]*')
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi there!"
    else:
        return "Hello, {0}!".format(name)

# Send messages
send_message("my name is David Copperfield")
send_message("call me Sia")
send_message("People call me Cassandra")


USER : my name is David Copperfield
BOT : Hello, David Copperfield!
USER : call me Sia
BOT : Hi there!
USER : People call me Cassandra
BOT : Hi there!


In [ ]:
# Word Vectors
# Apply ML to text
# Vectors can be units of characters, words or sentences
# word vectors try to represent meaning of words
# words which appear in similar context have similar vectors
# word2vec
# .similarity() Cosine

In [10]:
import spacy
import numpy as np
sentences = [' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning', ' what flights are available from pittsburgh to baltimore on thursday morning', ' what is the arrival time in san francisco for the 755 am flight leaving washington', ' cheapest airfare from tacoma to orlando', ' round trip fares from pittsburgh to philadelphia under 1000 dollars', ' i need a flight tomorrow from columbus to minneapolis', ' what kind of aircraft is used on a flight from cleveland to dallas', ' show me the flights from pittsburgh to los angeles on thursday', ' all flights from boston to washington', ' what kind of ground transportation is available in denver', ' show me the flights from dallas to san francisco', ' show me the flights from san diego to newark by way of houston', ' what is the cheapest flight from boston to bwi', ' all flights to baltimore after 6 pm', ' show me the first class fares from boston to denver', ' show me the ground transportation in denver', ' all flights from denver to pittsburgh leaving after 6 pm and before 7 pm', ' i need information on flights for tuesday leaving baltimore for dallas dallas to boston and boston to baltimore', ' please give me the flights from boston to pittsburgh on thursday of next week', ' i would like to fly from denver to pittsburgh on united airlines', ' show me the flights from san diego to newark', ' please list all first class flights on united from denver to baltimore', ' what kinds of planes are used by american airlines', " i'd like to have some information on a ticket from denver to pittsburgh and atlanta", " i'd like to book a flight from atlanta to denver", ' which airline serves denver pittsburgh and atlanta', " show me all flights from boston to pittsburgh on wednesday of next week which leave boston after 2 o'clock pm", ' atlanta ground transportation', ' i also need service from dallas to boston arriving by noon', ' show me the cheapest round trip fare from baltimore to dallas']
 


In [12]:
# Load the spacy model: nlp
nlp = spacy.load('en_core_web_md')

# Calculate the length of sentences
n_sentences = len(sentences)

# Calculate the dimensionality of nlp
embedding_dim = nlp.vocab.vectors_length
print('VOCAB_DIM:', embedding_dim)

# Initialize the array with zeros: X
X = np.zeros((n_sentences, embedding_dim))

# Iterate over the sentences
for idx, sentence in enumerate(sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X[idx, :] = doc.vector

VOCAB_DIM: 300


In [ ]:
# Intents & Classification
# use supervised learning, fit classifier, nearest neighbor in sklearn

In [16]:
# Intent classification with sklearn
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

df = pd.read_csv('Files/atis_intents.csv')
df.label = df.label.astype('category')
df.message = df.label.astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4978 entries, 0 to 4977
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   label    4978 non-null   category
 1   message  4978 non-null   string  
dtypes: category(1), string(1)
memory usage: 44.6 KB


In [18]:
def build_features(sentences):
    # Calculate the length of sentences
    n_sentences = len(sentences)

    # Calculate the dimensionality of nlp
    embedding_dim = nlp.vocab.vectors_length

    # Initialize the array with zeros: X
    X = np.zeros((n_sentences, embedding_dim))

    # Iterate over the sentences
    for idx, sentence in enumerate(sentences):
        # Pass each each sentence to the nlp object to create a document
        doc = nlp(sentence)
        # Save the document's .vector attribute to the corresponding row in X
        X[idx, :] = doc.vector

    return X

X = build_features(df['message'])

X_train, X_test, y_train, y_test = train_test_split(X, df.label, test_size=0.33, random_state=42)

print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('y_train', y_train.shape)
print('y_test', y_test.shape)

X_train (3335, 300)
X_test (1643, 300)
y_train (3335,)
y_test (1643,)


In [19]:
# Import SVC
from sklearn.svm import SVC

# Create a support vector classifier
clf = SVC(C=1)

# Fit the classifier using the training data
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Count the number of correct predictions
print(classification_report(y_test, y_pred))

                                          precision    recall  f1-score   support

                       atis_abbreviation       0.00      0.00      0.00        38
                           atis_aircraft       0.00      0.00      0.00        22
atis_aircraft#atis_flight#atis_flight_no       0.00      0.00      0.00         1
                            atis_airfare       0.00      0.00      0.00       141
                            atis_airline       0.00      0.00      0.00        49
                            atis_airport       0.00      0.00      0.00         4
                           atis_capacity       0.00      0.00      0.00         4
                               atis_city       0.00      0.00      0.00         8
                           atis_distance       0.00      0.00      0.00         4
                             atis_flight       0.76      1.00      0.86      1242
                atis_flight#atis_airfare       0.00      0.00      0.00         8
               

/Users/Sia/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Sia/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Sia/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Entity Extraction
# Using spaCy's entity recognizer
# spaCy's built-in entity recognizer to extract names, dates, and organizations from search queries.
# define a function called extract_entities(), which takes in a single argument message and returns a dictionary with the included entity types as keys, and the extracted entities as values. The included entity types are contained in a list called include_entities.

In [24]:
# Define included_entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in ents:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

print(extract_entities('friends called Mary who have worked at Google since 2010'))
print(extract_entities('Babak who graduated from UPC in 2022'))

{'DATE': '2010', 'ORG': 'Google', 'PERSON': 'Mary'}
{'DATE': '2022', 'ORG': 'UPC', 'PERSON': 'Babak'}


In [ ]:
# Assigning roles using spaCy's parser
# Create the document
doc = nlp("let's see that jacket in red and some blue jeans")

# Iterate over parents in parse tree until an item entity is found
def find_parent_item(word):
    # Iterate over the word's ancestors
    for parent in word.ancestors:
        # Check for an "item" entity
        if entity_type(parent) == "item":
            return parent.text
    return None

# For all color entities, find their parent item
def assign_colors(doc):
    # Iterate over the document
    for word in doc:
        # Check for "color" entities
        if entity_type(word) == "color":
            # Find the parent
            item = find_parent_item(word)
            print("item: {0} has color : {1}".format(item, word))

# Assign the colors
assign_colors(doc)